In [1]:

from __future__ import print_function
import numpy as np
import numpy.random
import numpy.linalg

#import sys
#import os
#lib_path = os.path.abspath('cppcd')
#sys.path.append(lib_path)
#import cppcd
#import lasso


def gen_normalized_gaussian_matrix(m, n):
  A = np.random.randn(m, n)
  # normalize
  col_norms = np.linalg.norm(A, axis=0)
  A = np.divide(A, col_norms)
  return A


def gen_sparse_rademacher_vector(n, k):
  x = np.zeros(n)
  coords = np.random.choice(n, k, replace=False)
  x[coords] = 1.0 - 2.0 * np.random.randint(0, 2, k)
  return x


def get_pyplot_data_iter(stats, opt=None):
  xvals = np.array([x for x, y in stats])
  yvals = np.array([y for x, y in stats])
  if not opt:
    opt = np.min(yvals)
  yvals -= opt
  yvals /= np.max(yvals)
  return xvals, yvals


def get_plot_data_time(progress, opt=None):
  xvals = [te.elapsed_time for te in progress]
  yvals = [te.current_objective_value for te in progress]
  if not opt:
    opt = np.min(yvals)
  yvals -= opt
  yvals /= np.max(yvals)
  return xvals, yvals

def get_plot_data_iter(progress, opt=None):
  yvals = [te.current_objective_value for te in progress]
  if not opt:
    opt = np.min(yvals)
  yvals -= opt
  yvals /= np.max(yvals)
  return yvals

import math
import numpy as np
import time
from sklearn import linear_model

import sys
import os
lib_path = os.path.abspath('../falconn/python_swig')
sys.path.append(lib_path)
import falconn

def soft_thresholding(x, threshold):
  if x >= threshold:
    return x - threshold
  elif x <= -threshold:
    return x + threshold
  else:
    return 0.0


def compute_objective_value(y, A, x, lambd):
  return 0.5 * np.linalg.norm(y - np.dot(A, x), ord=2)**2 + lambd * np.linalg.norm(x, ord=1)


def compute_x_error(xhat, x):
  return np.linalg.norm(xhat - x, ord=2) / np.linalg.norm(x, ord=2)


def meta_lasso(y, A, lambd, num_effective_passes, order_generator, verbose=False):
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y
  
  obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
  if verbose:
    print("  Initial objective value: {}".format(obj_value))
  
  stats = [(0.0, obj_value)]
  
  completed_effective_passes = 0.0
  while completed_effective_passes < num_effective_passes:
    coords = order_generator(y, A, x, r)
    for ii in coords:
      xiold = x[ii]
      thresh = lambd * inv_sq_col_norms[ii]
      val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
      xinew = soft_thresholding(val, thresh)
      r = r + A[:,ii] * (xiold - xinew)
      x[ii] = xinew
    completed_effective_passes += len(coords) / float(n)
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    stats.append((completed_effective_passes, obj_value))
    if verbose:
      print("  Objective value after {} effective passes: {}".format(completed_effective_passes, obj_value))

  return (x, stats)


def lasso_cyclic(y, A, lambd, num_passes, verbose=False):
  _, n = A.shape
  def cyclic_order_generator(*unused):
    return range(n)
  if verbose:
    print("Cyclic coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, cyclic_order_generator, verbose)


def lasso_randomiid(y, A, lambd, num_passes, verbose=False):
  _, n = A.shape
  def randiid_order_generator(*unused):
    return np.random.randint(n, size=n)
  if verbose:
    print("Random iid coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, randiid_order_generator, verbose)


def lasso_randomperm(y, A, lambd, num_passes, verbose=False):
  _, n = A.shape
  def randomperm_order_generator(*unused):
    return np.random.permutation(n)
  if verbose:
    print("Random permutation coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, randomperm_order_generator, verbose)

def lasso_maxip(y, A, lambd, num_passes, prefix_length=-1, verbose=False):
  _, n = A.shape
  if prefix_length < 0:
    prefix_length = n
  def maxip_order_generator(y, A, x, r):
    ips = np.dot(A.transpose(), r)
    indices = np.flipud(np.argsort(np.abs(ips)))
    print(indices[:10])
    return indices[:prefix_length]
  if verbose:
    print("Maximum inner product coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, maxip_order_generator, verbose)

def lasso_lsh(y, A, lambd, num_passes, prefix_length=-1, verbose=False):
  m, n = A.shape
  
  params = falconn.LSHConstructionParameters()
  params.dimension = m
  params.lsh_family = 'hyperplane'
  params.distance_function = 'negative_inner_product'
  params.k = int(math.floor(math.log(n, 2)))
  params.l = 10
  params.seed = 833840234
  AT = np.transpose(A)
  AT = AT.astype(np.float32)
  table = falconn.construct_table_dense_float(AT, params)
  table.set_num_probes(2 ** params.k * params.l)

  if prefix_length < 0:
    prefix_length = n

  def lsh_order_generator(y, A, x, r):
    r2 = r.astype(np.float32)
    res1 = table.get_unique_candidates(r2)
    res2 = table.get_unique_candidates(-r2)
    seq = [index for pair in zip(res1, res2) for index in pair]
    print(seq[:10])
    return seq[:prefix_length]
  if verbose:
    print("LSH coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, lsh_order_generator, verbose)

  

def lasso_cyclic_explicit(y, A, lambd, num_passes, verbose=False):
  if verbose:
    print("Cyclic coordinate descent Lasso solver")
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y

  if verbose:
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    print("  Initial objective value: {}".format(obj_value))


  for cur_cycle in range(num_passes):
    num_nonzero = 0
    for ii in range(n):
      xiold = x[ii]
      thresh = lambd * inv_sq_col_norms[ii]
      val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
      xinew = soft_thresholding(val, thresh)
      r = r + A[:,ii] * (xiold - xinew)
      x[ii] = xinew
      if abs(xinew) > 1e-6:
        num_nonzero += 1

    if verbose:
      obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
      print("  Current objective value: {}  ({} nonzero coordinates)".format(obj_value, num_nonzero))
  return x


def lasso_random_explicit(y, A, lambd, num_steps, verbose=False):
  if verbose:
    print("Random coordinate descent Lasso solver")
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y

  if verbose:
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    print("  Initial objective value: {}".format(obj_value))

  for cur_step in range(num_steps):
    ii = np.random.randint(0, n)
    xiold = x[ii]
    thresh = lambd * inv_sq_col_norms[ii]
    val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
    xinew = soft_thresholding(val, thresh)
    r = r + A[:,ii] * (xiold - xinew)
    x[ii] = xinew
    
    if verbose and cur_step % n == 0 and cur_step > 0:
      obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
      print("  Current objective value: {}".format(obj_value))
  return x


def lasso_steepest(y, A, lambd, num_steps, verbose=False):
  if verbose:
    print("Steepest coordinate descent Lasso solver")
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y
  r_values = []

  if verbose:
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    print("  Initial objective value: {}".format(obj_value))

  for cur_step in range(num_steps):
    derivatives = np.abs(np.dot(np.transpose(A), r / np.linalg.norm(r, ord=2)))
    r_values.append(r / np.linalg.norm(r, ord=2))
    ii = np.argmax(derivatives)
    # Alternative coordinate selection methods (e.g., largest progress)
    #  #derivatives = np.abs(np.dot(np.transpose(A), r) - lambd * np.sign(x))
    #print("  Largest inner product: {}".format(derivatives[ii]))
    #ii = -1
    #best_abs = 0.0
    #for jj in range(n):
    #  xjold = x[jj]
    #  thresh = lambd * inv_sq_col_norms[jj]
    #  val = np.dot(A[:,jj], r) * inv_sq_col_norms[jj] + xjold
    #  xjnew = soft_thresholding(val, thresh)
    #  if abs(xjold - xjnew) > best_abs:
    #    ii = jj
    #    best_abs = abs(xjold - xjnew)
      
    xiold = x[ii]
    thresh = lambd * inv_sq_col_norms[ii]
    val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
    xinew = soft_thresholding(val, thresh)
    #print("    diff = {}  thresh = {}  xiold = {}  xinew = {}".format(np.dot(A[:,ii], r) * inv_sq_col_norms[ii], thresh, xiold, xinew))
    r = r + A[:,ii] * (xiold - xinew)
    x[ii] = xinew
   
    if verbose: 
      obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
      #print("  Actually largest inner product: {} (coord {})".format(np.max(derivatives), np.argmax(derivatives)))
      print("  Largest inner product: {:.3} (coord {})  Average inner product: {:.4}  Current objective value: {}".format(derivatives[ii], ii, np.mean(derivatives), obj_value))
  return x, r_values


# From https://github.com/adarshvjois/LASSO/blob/master/HW2/shooting_algorithm.py
def lasso_cyclic_reference(y, X, lambda_reg, num_iter, w_init=None):
    """
    X - Training set
    y - training values to classify/do regression on.
    w_init - my initial guess for the weights
    lambda_reg - hyper parameter
    
    returns the final weight vector.
    """
    lambda_reg *= 2   # added by Ludwig
    #TODO add a way in which I can include tolerance.
    if w_init is not None:
        w = w_init 
    else:
        w = np.zeros(X.shape[1])
    D = X.shape[1]

    XX2 = np.dot(X.T, X) * 2
    Xy2 = np.dot(X.T, y) * 2
    i = 0
    while i < num_iter:
        i += 1
        for j in range(D):
            c_j = Xy2[j] - XX2[j, :].dot(w) + XX2[j, j] * w[j]
            a_j = XX2[j, j]
            
            if c_j < -lambda_reg:
                w[j] = (c_j + lambda_reg) / a_j
            elif c_j > lambda_reg:
                w[j] = (c_j - lambda_reg) / a_j
            else:
                w[j] = 0
    return w


def lasso_sklearn(y, A, lambd, tol=1e-5, max_iter=100000, selection='random'):
  m, n = A.shape
  lambda_prime = lambd / n

  lasso = linear_model.Lasso(alpha=lambda_prime, copy_X=True, normalize=False, positive=False, tol=tol, max_iter=max_iter, fit_intercept=False, selection=selection)
  lasso.fit(A, y)
  return lasso.coef_


In [6]:
np.random.seed(5673932)
algo_random_seed = np.random.randint(0, 1000000000)
n = 10000
m = 2000
k = 200

# Generate data
A = gen_normalized_gaussian_matrix(m, n)
x = gen_sparse_rademacher_vector(n, k)
y = np.dot(A, x)

#lambd = 0.01 * math.log(n, 2) / m
lambd =  math.sqrt(2.0 * math.log(n, 2) / m)
print("lambda = {}".format(lambd))
num_passes = 20

lambda = 0.115272340045


In [7]:
x_lsh, stats_lsh = lasso_lsh(y, A, lambd, num_passes, verbose=True)

LSH coordinate descent Lasso solver
  Initial objective value: 97.5539232301
[6269, 7686, 4117, 7689, 3449, 90, 9321, 918, 3284, 3286]
  Objective value after 1.0 effective passes: 48.4698471426
[8172, 9816, 1831, 9193, 3876, 7067, 5082, 6610, 3421, 3287]
  Objective value after 2.0 effective passes: 35.2043914999
[4450, 6124, 7473, 4588, 573, 5290, 4586, 6299, 1558, 9137]
  Objective value after 3.0 effective passes: 29.8173234311
[8163, 4085, 7473, 2545, 2746, 516, 7492, 872, 2782, 9803]
  Objective value after 4.0 effective passes: 26.5254800898
[7962, 9518, 9356, 6657, 7388, 8620, 6817, 5317, 2866, 5180]
  Objective value after 5.0 effective passes: 24.6546313731
[4358, 6125, 9224, 6050, 8250, 3075, 4769, 5687, 9126, 6385]
  Objective value after 6.0 effective passes: 23.335961528
[7962, 9518, 491, 6657, 4354, 8620, 9356, 5317, 4358, 3545]
  Objective value after 7.0 effective passes: 22.4807063936
[7962, 5180, 9356, 4478, 6817, 7917, 7659, 9518, 2437, 6657]
  Objective value after

In [2]:
bool('False')

True

In [3]:
bool('True')

True

In [8]:
bool('f')

True

In [11]:
def str2bool(v):
  return v.lower() in ("yes", "true", "t", "1")
str2bool('true')

True

In [11]:
import numpy as np
ii = np.random.randint(10)

In [12]:
ii

1

In [1]:
a = [1,3,5]

In [5]:
for b in a:
  print(b)
  if(b <5):
    b = 12
  print(b)

1
12
3
12
5
5
